# Checkdata 1

In [ ]:
import pandas as pd
import re
import numpy as np

# ==========================================
# 1. LOAD & PREPARE DATA
# ==========================================
try:
    df = pd.read_parquet("./data_marker/kb_tables.parquet")
    print(f"📦 Loaded {len(df)} rows from kb_tables.parquet")
except FileNotFoundError:
    print("❌ kb_tables.parquet not found.")
    df = pd.DataFrame()

if not df.empty:
    print("🔄 Pivoting data to reconstruct table rows...")
    wide_df = df.pivot_table(
        index=['doc_name', 'table_id', 'row_id'], 
        columns='column', 
        values='value_str', 
        aggfunc='first'
    ).reset_index()
    print(f"✅ Reconstructed {len(wide_df)} unique table rows.")
else:
    wide_df = pd.DataFrame()

# ==========================================
# 2. EXTRACTION LOGIC (V3)
# ==========================================
def _norm(s):
    return str(s).lower().strip()

def get_row_text(row):
    return " ".join([str(x) for x in row.values if x is not None]).lower()

def extract_metric_series(target_label_keywords, year_cols=True):
    matches = []
    if wide_df.empty: return pd.DataFrame()

    all_cols = wide_df.columns.astype(str)
    year_columns = [c for c in all_cols if re.match(r'^\d{4}$', c)]
    
    for idx, row in wide_df.iterrows():
        doc_name = row['doc_name']
        row_text = get_row_text(row)
        
        # --- STRATEGY A: Financials (Years) ---
        if year_cols:
            if any(k in row_text for k in target_label_keywords):
                data = {}
                for y in year_columns:
                    if y in row and pd.notna(row[y]):
                        val_str = str(row[y]).replace(',', '')
                        try:
                            data[int(y)] = float(val_str)
                        except ValueError:
                            pass
                if data:
                    label_guess = "Unknown"
                    for val in row.values:
                        if pd.notna(val) and any(k in _norm(val) for k in target_label_keywords):
                            label_guess = val
                            break
                    matches.append({"Doc": doc_name, "Label": label_guess, "Data": data})

        # --- STRATEGY B: NIM/Metrics (Quarters) ---
        else:
            valid_items = row.dropna().to_dict()
            q_candidates = [k for k in valid_items.keys() if 'quarter' in str(k).lower()]
            q_candidates.sort(key=len)
            
            if q_candidates:
                q_col = q_candidates[0]
                quarter_val = valid_items[q_col]
                for col_name, cell_val in valid_items.items():
                    if col_name == q_col: continue
                    # Check column header for keyword
                    if any(k in _norm(col_name) for k in target_label_keywords):
                        matches.append({
                            "Doc": doc_name, 
                            "Label": col_name, 
                            "Data": {quarter_val: cell_val}
                        })

    return pd.DataFrame(matches)

# ==========================================
# 3. RUN EXTRACTORS (RAW DATA DUMP)
# ==========================================
print("\n" + "="*60)
print("🔎 RAW DATA VERIFICATION (NO CALCULATIONS)")
print("="*60)

# --- 1. OPERATING EXPENSES ---
print("\n🔹 RAW PULL: Operating Expenses")
df_opex = extract_metric_series(["total expenses", "operating expenses"], year_cols=True)

if not df_opex.empty:
    # Filter to show only meaningful rows (>= 2 data points)
    df_opex_clean = df_opex[df_opex['Data'].apply(len) >= 2]
    for _, r in df_opex_clean.iterrows():
        # Sort data keys for cleaner viewing
        sorted_data = dict(sorted(r['Data'].items()))
        print(f"📄 {r['Doc']} | Label: {r['Label']}")
        print(f"   📊 {sorted_data}")
        print("-" * 20)
else:
    print("❌ No Opex data found.")

# --- 2. OPERATING INCOME ---
print("\n🔹 RAW PULL: Operating Income")
df_inc = extract_metric_series(["total income", "operating income"], year_cols=True)

if not df_inc.empty:
    df_inc_clean = df_inc[df_inc['Data'].apply(len) >= 2]
    for _, r in df_inc_clean.iterrows():
        sorted_data = dict(sorted(r['Data'].items()))
        print(f"📄 {r['Doc']} | Label: {r['Label']}")
        print(f"   📊 {sorted_data}")
        print("-" * 20)
else:
    print("❌ No Income data found.")

# --- 3. NET INTEREST MARGIN ---
print("\n🔹 RAW PULL: Net Interest Margin (Quarterly)")
df_nim = extract_metric_series(["nim", "net interest margin"], year_cols=False)

if not df_nim.empty:
    # Group by Doc to show the timeline extracted per document
    for doc, group in df_nim.groupby("Doc"):
        timeline = {}
        for _, r in group.iterrows():
            # Only keep keys that look like quarters (contain 'q')
            valid_q = {k:v for k,v in r['Data'].items() if 'q' in str(k).lower()}
            timeline.update(valid_q)
        
        if timeline:
            sorted_timeline = dict(sorted(timeline.items()))
            print(f"📄 {doc}")
            print(f"   📈 Raw Series: {sorted_timeline}")
            print("-" * 20)
else:
    print("❌ No NIM data found.")

# Checkdata 2

In [ ]:
import pandas as pd
import re
import numpy as np

# ==========================================
# 1. LOAD & PREPARE DATA
# ==========================================
try:
    df = pd.read_parquet("./data_marker/kb_tables.parquet")
    print(f"📦 Loaded {len(df)} rows from kb_tables.parquet")
except FileNotFoundError:
    print("❌ kb_tables.parquet not found.")
    df = pd.DataFrame()

if not df.empty:
    print("🔄 Pivoting data to reconstruct table rows...")
    wide_df = df.pivot_table(
        index=['doc_name', 'table_id', 'row_id'], 
        columns='column', 
        values='value_str', 
        aggfunc='first'
    ).reset_index()
    print(f"✅ Reconstructed {len(wide_df)} unique table rows.")
else:
    wide_df = pd.DataFrame()

# ==========================================
# 2. EXTRACTION LOGIC (V3)
# ==========================================
def _norm(s):
    return str(s).lower().strip()

def get_row_text(row):
    return " ".join([str(x) for x in row.values if x is not None]).lower()

def extract_metric_series(target_label_keywords, year_cols=True):
    matches = []
    if wide_df.empty: return pd.DataFrame()

    all_cols = wide_df.columns.astype(str)
    year_columns = [c for c in all_cols if re.match(r'^\d{4}$', c)]
    
    for idx, row in wide_df.iterrows():
        doc_name = row['doc_name']
        row_text = get_row_text(row)
        
        # --- STRATEGY A: Financials (Years) ---
        if year_cols:
            if any(k in row_text for k in target_label_keywords):
                data = {}
                for y in year_columns:
                    if y in row and pd.notna(row[y]):
                        val_str = str(row[y]).replace(',', '')
                        try:
                            data[int(y)] = float(val_str)
                        except ValueError:
                            pass
                if data:
                    label_guess = "Unknown"
                    for val in row.values:
                        if pd.notna(val) and any(k in _norm(val) for k in target_label_keywords):
                            label_guess = val
                            break
                    matches.append({"Doc": doc_name, "Label": label_guess, "Data": data})

        # --- STRATEGY B: NIM/Metrics (Quarters) ---
        else:
            valid_items = row.dropna().to_dict()
            q_candidates = [k for k in valid_items.keys() if 'quarter' in str(k).lower()]
            q_candidates.sort(key=len)
            
            if q_candidates:
                q_col = q_candidates[0]
                quarter_val = valid_items[q_col]
                for col_name, cell_val in valid_items.items():
                    if col_name == q_col: continue
                    if any(k in _norm(col_name) for k in target_label_keywords):
                        matches.append({
                            "Doc": doc_name, 
                            "Label": col_name, 
                            "Data": {quarter_val: cell_val}
                        })

    return pd.DataFrame(matches)

# ==========================================
# 3. RUN EXTRACTORS
# ==========================================
print("\n🔎 Extracting raw data...")
df_opex = extract_metric_series(["total expenses", "operating expenses"], year_cols=True)
df_inc = extract_metric_series(["total income", "operating income"], year_cols=True)
df_nim = extract_metric_series(["nim", "net interest margin"], year_cols=False)

print(f"   found {len(df_opex)} Opex rows")
print(f"   found {len(df_inc)} Income rows")
print(f"   found {len(df_nim)} NIM rows")

# ==========================================
# 4. GENERATE BENCHMARK ANSWERS (FIXED)
# ==========================================
print("\n" + "="*60)
print("📝 FINAL BENCHMARK OUTPUTS")
print("="*60)

# Helper to merge multiple rows into one timeline (taking MAX value to find Group Total)
def merge_financial_series(df_source):
    master_timeline = {}
    if df_source.empty: return master_timeline
    
    for _, row in df_source.iterrows():
        for year, val in row['Data'].items():
            # If we have multiple values for 2023 (e.g. Segment vs Group), take the larger one
            current_max = master_timeline.get(year, 0)
            if val > current_max:
                master_timeline[year] = val
    return master_timeline

# --- ANSWER 1: NIM TREND ---
print("\n🔹 1. Gross Margin / NIM Trend (Last 5 Quarters)")
if not df_nim.empty:
    timeline = {}
    for _, r in df_nim.iterrows():
        # Clean keys
        clean_q = {k:v for k,v in r['Data'].items() if 'q' in str(k).lower()}
        timeline.update(clean_q)
    
    def q_sorter(q):
        m = re.match(r'([1-4])q(\d{2})', str(q).lower())
        if m: return int(f"20{m.group(2)}{m.group(1)}")
        return 0

    if timeline:
        sorted_qs = sorted(timeline.keys(), key=q_sorter)
        display_qs = sorted_qs[-5:] if len(sorted_qs) > 5 else sorted_qs
        
        print(f"| Quarter | Group NIM (%) |")
        print(f"|:-------:|:-------------:|")
        for q in display_qs:
            print(f"| {q:<7} | {timeline[q]:<13} |")
    else:
        print("❌ No valid quarterly keys found.")
else:
    print("❌ No NIM data found.")


# --- ANSWER 2: OPEX YoY ---
print("\n🔹 2. Operating Expenses (3 Years YoY)")
opex_data = merge_financial_series(df_opex)

if opex_data:
    print(f"| Year | Opex ($m) | YoY Change (%) |")
    print(f"|:----:|:---------:|:--------------:|")
    
    years = sorted(opex_data.keys())[-3:] # Last 3 years available
    for i, y in enumerate(years):
        val = opex_data[y]
        yoy_str = "-"
        
        if (y - 1) in opex_data:
            prev_val = opex_data[y-1]
            if prev_val != 0:
                pct = ((val - prev_val) / prev_val) * 100
                yoy_str = f"{pct:+.1f}%"
        
        print(f"| {y:<4} | {val:<9,.1f} | {yoy_str:<14} |")
else:
    print("❌ No Opex data found.")


# --- ANSWER 3: EFFICIENCY RATIO ---
print("\n🔹 3. Operating Efficiency Ratio (Opex ÷ Income)")
inc_data = merge_financial_series(df_inc)

if opex_data and inc_data:
    common_years = sorted(set(opex_data.keys()) & set(inc_data.keys()))[-3:]
    
    print(f"| Year | Opex ($m) | Income ($m) | Efficiency Ratio (%) |")
    print(f"|:----:|:---------:|:-----------:|:--------------------:|")
    
    for y in common_years:
        opex = opex_data[y]
        inc = inc_data[y]
        ratio = (opex / inc) * 100 if inc != 0 else 0
        print(f"| {y:<4} | {opex:<9,.1f} | {inc:<11,.1f} | {ratio:<20.1f} |")
    
    print(f"\n*Calculation: (Operating Expenses / Total Income) * 100")
else:
    print("❌ Insufficient data (need both Opex and Income).")